# Introduction 
Dans ce notebook, nous chargeons et lisons les données météorologiques sur la période de janvier 2005 à janvier 2025 via la librairie Meteostat. Les données chargées sont journalières

In [0]:
# COMMAND ----------
# Installation de la librairie Meteostat
#%pip install meteostat
# Redémarrage du noyau pour garantir que les imports fonctionnent
#dbutils.library.restartPython()

# Chargement des données météorologiques journalières du Sénégal
### Source
_Meteostat (stations météorologiques réelles)_
### Couverture
_14 régions administratives du Sénégal_
### Objectif
_Collecter des données journalières météorologiques brutes à l’échelle nationale
afin de préparer une analyse Big Data avec Apache Spark._

1. IMPORTS ET CONFIGURATION SPARK


In [0]:
# LES COMMANDES A CHARGER----------
import pandas as pd
from meteostat import Point, Daily
from datetime import datetime
from pyspark.sql import SparkSession

In [0]:
# Initialisation de Spark sur Databricks
spark = SparkSession.builder.appName("Meteo_Senegal_Raw_Ingestion").getOrCreate()

3. PARAMÈTRES GLOBAUX

In [0]:
# COMMAND ----------
# Période d'étude (20 ans de données journalières)
START_DATE = datetime(2005, 1, 1)
END_DATE   = datetime(2025, 1, 1)

4. MÉTADONNÉES DES STATIONS (14 RÉGIONS)

In [0]:
# COMMAND ----------
# Stations météo représentant les régions du Sénégal
STATIONS_SENEGAL = [
    ("Dakar", 14.7303, -17.4439),
    ("Diourbel", 14.65, -16.25),
    ("Fatick", 14.35, -16.42),
    ("Kaffrine", 14.10, -15.55),
    ("Kaolack", 14.1333, -16.0333),
    ("Kédougou", 12.55, -12.18),
    ("Kolda", 12.88, -14.95),
    ("Louga", 15.6167, -16.2167),
    ("Matam", 15.60, -13.28),
    ("Saint-Louis", 16.0597, -16.4678),
    ("Sédhiou", 12.7167, -15.5667),
    ("Tambacounda", 13.7333, -13.6667),
    ("Thiès", 14.80, -16.90),
    ("Ziguinchor", 12.55, -16.27)
]

5. FONCTION D’INGESTION (LOGIQUE RÉUTILISABLE)

In [0]:
# COMMAND ----------
def collect_weather_daily(region_name, latitude, longitude, start_date, end_date):
    """
    Collecte des données météorologiques journalières brutes
    pour une station donnée via Meteostat.
    """
    try:
        point = Point(latitude, longitude)
        data = Daily(point, start_date, end_date)
        df = data.fetch()
        
        if df.empty:
            print(f" {region_name} : aucune donnée disponible.")
            return None
        
        df.reset_index(inplace=True)
        df["region"] = region_name
        
        print(f" {region_name} : {len(df)} observations collectées.")
        return df
    
    except Exception as e:
        print(f" {region_name} : erreur de collecte → {e}")
        return None

6. BOUCLE D’INGESTION NATIONALE

In [0]:
# COMMAND ----------
print(
    f"Collecte des données météorologiques journalières "
    f"pour le Sénégal ({START_DATE.year}–{END_DATE.year})"
)

pandas_frames = []

for region, lat, lon in STATIONS_SENEGAL:
    df_region = collect_weather_daily(
        region, lat, lon, START_DATE, END_DATE
    )
    
    if df_region is not None:
        pandas_frames.append(df_region)

Collecte des données météorologiques journalières pour le Sénégal (2005–2025)
 Dakar : 7306 observations collectées.


 Diourbel : 7305 observations collectées.
 Fatick : aucune donnée disponible.
 Kaffrine : aucune donnée disponible.
 Kaolack : 7303 observations collectées.
 Kédougou : 7303 observations collectées.
 Kolda : 7303 observations collectées.
 Louga : aucune donnée disponible.
 Matam : 7303 observations collectées.
 Saint-Louis : 7302 observations collectées.
 Sédhiou : aucune donnée disponible.
 Tambacounda : 7303 observations collectées.
 Thiès : aucune donnée disponible.
 Ziguinchor : 7303 observations collectées.


Le résultat précedent montre que les région de Fatick, Thies, Louga, Kaffrine et Sédhiou n'ont aucunes données disponibles. Nous l'avons vérifier en vérifiant les zones couvertes par le Satellite meteastat en utilisant les commandes ci-dessous : 

from meteostat import Stations
stations = Stations()
stations = stations.region('SN')  
stations_df = stations.fetch()
stations_df.head()


7. CONSOLIDATION & CONVERSION SPARK

In [0]:
# COMMAND ----------
# Fusion nationale (pandas → big table)
df_weather_pandas = pd.concat(pandas_frames, ignore_index=True)

In [0]:
# Conversion en DataFrame Spark
df_weather_spark = spark.createDataFrame(df_weather_pandas)

8. VALIDATION DES DONNÉES BRUTES

In [0]:
# COMMAND ----------
print("="*60)
print("DONNÉES MÉTÉO JOURNALIÈRES BRUTES – NIVEAU NATIONAL")
print(f"Nombre total d'observations : {df_weather_spark.count()}")
print("Schéma de la table :")
df_weather_spark.printSchema()

DONNÉES MÉTÉO JOURNALIÈRES BRUTES – NIVEAU NATIONAL
Nombre total d'observations : 65731
Schéma de la table :
root
 |-- time: timestamp (nullable = true)
 |-- tavg: double (nullable = true)
 |-- tmin: double (nullable = true)
 |-- tmax: double (nullable = true)
 |-- prcp: double (nullable = true)
 |-- snow: double (nullable = true)
 |-- wdir: double (nullable = true)
 |-- wspd: double (nullable = true)
 |-- wpgt: double (nullable = true)
 |-- pres: double (nullable = true)
 |-- tsun: double (nullable = true)
 |-- region: string (nullable = true)



Le résultat précedent montre que notre base a 65731 observations

In [0]:
display(df_weather_spark)

time,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun,region
2005-01-01T00:00:00.000Z,22.6,18.5,28.0,null,null,null,16.4,null,null,null,Dakar
2005-01-02T00:00:00.000Z,21.5,18.8,24.0,null,null,null,17.0,null,null,null,Dakar
2005-01-03T00:00:00.000Z,20.6,null,null,null,null,null,19.3,null,null,null,Dakar
2005-01-04T00:00:00.000Z,20.5,null,24.3,null,null,null,19.2,null,null,null,Dakar
2005-01-05T00:00:00.000Z,20.1,18.0,23.0,0.0,null,null,16.2,null,null,null,Dakar
2005-01-06T00:00:00.000Z,20.7,18.0,null,0.0,null,null,15.0,null,null,null,Dakar
2005-01-07T00:00:00.000Z,22.1,19.0,null,0.0,null,null,14.6,null,null,null,Dakar
2005-01-08T00:00:00.000Z,22.1,19.9,25.0,0.0,null,null,21.2,null,null,null,Dakar
2005-01-09T00:00:00.000Z,21.6,19.0,25.0,null,null,null,19.0,null,null,null,Dakar
2005-01-10T00:00:00.000Z,23.8,20.0,32.0,0.0,null,null,16.0,null,null,null,Dakar


In [0]:
# =======================================
# Sauvegarde des données météo brutes
# =======================================

(df_weather_spark
 .write
 .mode("overwrite")
 .format("delta")
 .saveAsTable("weather_senegal_raw")
)

In [0]:
spark.sql("SHOW TABLES").show()

+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
| default|agriculture_weath...|      false|
| default|agriculture_weath...|      false|
| default|data_economics_clean|      false|
| default|data_final_agri_c...|      false|
| default|  depenses_agricoles|      false|
| default|depenses_agricole...|      false|
| default|   donnees_agricoles|      false|
| default|     ny_taxi_summary|      false|
| default|            pib_brut|      false|
| default|           pib_clean|      false|
| default| senegal_weather_raw|      false|
| default|students_performance|      false|
| default| weather_senegal_raw|      false|
+--------+--------------------+-----------+



## Conclusion
Les données météorologiques journalières brutes ont été collectées à partir
de stations météorologiques réelles couvrant 9 régions sur les 14 régions du Sénégal.
Ces données constituent la base du processus ETL et seront nettoyées,
harmonisées et agrégées dans les étapes suivantes notamment dans le notebook 03. Quality_weather_data du dossier data processing